#### Given an array of integers, find the first missing positive integer in linear time and constant space. In other words, find the lowest positive integer that does not exist in the array. The array can contain duplicates and negative numbers as well.
#### For example, the input [3, 4, -1, 1] should give 2. The input [1, 2, 0] should give 3.
#### You can modify the input array in-place.


In [1]:
from timeit import time
from random import shuffle

In [2]:
def timing(f):
    def wrapper(*args):
        start_t = time.time()
        ret = f(*args)
        print("@ {} takes {:.10f} ms".format(f, (time.time()-start_t)*1000))
        return ret
    return wrapper

In [3]:
X_1 = [3, 4, -1, 1] # should give 2
X_2 = [0, 1, 2] # should give 3

X_3 = [x for x in range(-5000, 100000)]
X_3.remove(100) # should give 100
shuffle(X_3)

In [4]:
@timing
def sorted_search(X):
    X = sorted(X) # O(n log n) --> Timsort algorithm
    n = 1
    def search(X):
        nonlocal n # to access n in outer stack
        for el in X:   # O(n)
            if el == n:
                n = n + 1
        return n
    search(X)
    return n
    # O(2n * log n) ~ O(n log n)
    # X : NO LINEAR TIME
    # V : CONSTANT SPACE

In [5]:
@timing
def hashed_search(X):
    X = set(X) # set in python uses hash table
               # creation takes O(n) in space dimension
    n = 1
    def search(X):
        nonlocal n
        for el in X:
            if el == n:
                n = n + 1
    search(X)
    return n
    # V : linear time
    # X : extra space

In [6]:
assert sorted_search(X_1) == 2
assert sorted_search(X_2) == 3
assert sorted_search(X_3) == 100

@ <function sorted_search at 0x0000021A4BCB8DC8> takes 0.0000000000 ms
@ <function sorted_search at 0x0000021A4BCB8DC8> takes 0.0000000000 ms
@ <function sorted_search at 0x0000021A4BCB8DC8> takes 46.8745231628 ms


In [7]:
assert hashed_search(X_1) == 2
assert hashed_search(X_2) == 3
assert hashed_search(X_3) == 100

@ <function hashed_search at 0x0000021A4C021048> takes 0.0000000000 ms
@ <function hashed_search at 0x0000021A4C021048> takes 0.0000000000 ms
@ <function hashed_search at 0x0000021A4C021048> takes 18.9478397369 ms


<p>Using set makes function a little bit fast:
<br>HASHED SEARCH: ~ 19 ms
<br>SORTED SEARCH: ~ 40 ms

In [8]:
@timing
def recursive_search(X):
    def n(a, X):
        for e in X:
            if e == a:
                return n(a+1, X)
        return a
    return n(1, X)

@timing
def recursive_search_with_split(X):
    def split(X):
        '''
        Utility that puts all non-positive numbers on left side of the array
        '''
        j = 0
        for i in range(len(X)):
            if X[i] <= 0:
                X[i], X[j] = X[j], X[i]
                j += 1
        return j
    def n(a, X):
        for e in X:
            if e == a:
                return n(a+1, X)
        return a
    return n(1, X[split(X):])

In [9]:
assert recursive_search(X_1) == 2
assert recursive_search(X_2) == 3
assert recursive_search(X_3) == 100

@ <function recursive_search at 0x0000021A4C021798> takes 0.0000000000 ms
@ <function recursive_search at 0x0000021A4C021798> takes 0.0000000000 ms
@ <function recursive_search at 0x0000021A4C021798> takes 225.3103256226 ms


In [10]:
assert recursive_search_with_split(X_1) == 2
assert recursive_search_with_split(X_2) == 3
assert recursive_search_with_split(X_3) == 100

@ <function recursive_search_with_split at 0x0000021A4C021558> takes 0.0000000000 ms
@ <function recursive_search_with_split at 0x0000021A4C021558> takes 0.0000000000 ms
@ <function recursive_search_with_split at 0x0000021A4C021558> takes 225.4338264465 ms


In [11]:
@timing
def check_presence(X):
    X = X.copy() # in order to not modify the original one -> this makes the algo non constant in space -> comment this line!
    neg = 0  
    def split(X):
        '''
        Utility that puts all non-positive numbers on left side of the array
        '''
        j = 0
        for i in range(len(X)):
            if X[i] <= 0:
                X[i], X[j] = X[j], X[i]
                j += 1
        return j
    def find_positive(X):
        '''
        Supposing X's elements positive
        '''
        if max(X) == len(X):
            return max(X)+1
        
        for i, num in enumerate(X):
            num = abs(num)
            if (num - 1) < len(X):
                X[num - 1] = -1
        
        for i, num in enumerate(X):
            if num > 0:
                return i + 1
    j = split(X)
    return find_positive(X[j:])

In [12]:
assert check_presence(X_1) == 2
assert check_presence(X_2) == 3
X_3 = [x for x in range(-5000, 100000)]
X_3.remove(521) # should give 521
assert check_presence(X_3) == 521

@ <function check_presence at 0x0000021A4C021A68> takes 0.0000000000 ms
@ <function check_presence at 0x0000021A4C021A68> takes 0.0000000000 ms
@ <function check_presence at 0x0000021A4C021A68> takes 56.8530559540 ms
